In [3]:
import pandas as pd

In [4]:
pd.options.display.max_rows = 4000

In [5]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [6]:
train = pd.read_csv("data/feature_selection.csv")

In [7]:
train.head()

,ID,age,class,A.c1.1,A.c1.2,A.c1.3,A.c1.4,A.c1.5,A.c2.1,A.c2.2,...,J.c8.1,J.c8.2,J.c8.3,J.c8.4,J.c8.5,J.c9.1,J.c9.2,J.c9.3,J.c9.4,J.c9.5
0,5560,3,typ,2.238984,3.238984,4.238984,5.238984,6.238984,2.539386,3.539386,...,2.330630,3.330630,4.330630,5.330630,6.330630,0.105146,1.105146,2.105146,3.105146,4.105146
1,4694,3,typ,1.490947,2.490947,3.490947,4.490947,5.490947,0.692924,1.692924,...,0.033946,1.033946,2.033946,3.033946,4.033946,-0.921489,0.078511,1.078511,2.078511,3.078511
2,6449,3,typ,1.828413,2.828413,3.828413,4.828413,5.828413,2.995978,3.995978,...,-0.309544,0.690456,1.690456,2.690456,3.690456,1.838188,2.838188,3.838188,4.838188,5.838188
3,3008,3,asd,1.930039,2.930039,3.930039,4.930039,5.930039,2.698195,3.698195,...,0.727438,1.727438,2.727438,3.727438,4.727438,2.793029,5.793029,10.793029,17.793029,26.793029
4,3863,3,typ,2.272464,3.272464,4.272464,5.272464,6.272464,1.539144,2.539144,...,2.168858,3.168858,4.168858,5.168858,6.168858,-0.938000,0.062000,1.062000,2.062000,3.062000


In [8]:
train.columns

Index(['ID', 'age', 'class', 'A.c1.1', 'A.c1.2', 'A.c1.3', 'A.c1.4', 'A.c1.5',
       'A.c2.1', 'A.c2.2',
       ...
       'J.c8.1', 'J.c8.2', 'J.c8.3', 'J.c8.4', 'J.c8.5', 'J.c9.1', 'J.c9.2',
       'J.c9.3', 'J.c9.4', 'J.c9.5'],
      dtype='object', length=453)

In [12]:
train["class"].nunique()

2

In [9]:
import pandas as pd
import numpy as np
import random
from itertools import combinations

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.random_projection import GaussianRandomProjection
from sklearn.random_projection import SparseRandomProjection
from sklearn.decomposition import PCA, FastICA
from sklearn.decomposition import TruncatedSVD
from sklearn.cluster import FeatureAgglomeration

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import Ridge

from sklearn.metrics import r2_score
from sklearn.preprocessing import LabelEncoder
import pickle

import warnings
warnings.filterwarnings('ignore')

random.seed(1729)

In [14]:
y_train = train['y']
train = train.drop('y', axis=1)

for c in train.columns:
    if train[c].dtype == 'object':
        lbl = LabelEncoder()
        lbl.fit(list(train[c].values))
        train[c] = lbl.transform(list(train[c].values))

LabelEncoder()

In [16]:
n_comp = 12

# tSVD
tsvd = TruncatedSVD(n_components=n_comp, random_state=420)
tsvd_results_train = tsvd.fit_transform(train)

# PCA
pca = PCA(n_components=n_comp, random_state=420)
pca2_results_train = pca.fit_transform(train)

# ICA
ica = FastICA(n_components=n_comp, random_state=420)
ica2_results_train = ica.fit_transform(train)

# GRP
grp = GaussianRandomProjection(n_components=n_comp, eps=0.1, random_state=420)
grp_results_train = grp.fit_transform(train)

# SRP
srp = SparseRandomProjection(n_components=n_comp, dense_output=True, random_state=420)
srp_results_train = srp.fit_transform(train)

# FAG
fag = FeatureAgglomeration(n_clusters=n_comp, linkage='ward')
fag_results_train = fag.fit_transform(train)


In [17]:
dim_reds = list()
train_pca = pd.DataFrame()
train_ica = pd.DataFrame()
train_tsvd = pd.DataFrame()
train_grp = pd.DataFrame()
train_srp = pd.DataFrame()
train_fag = pd.DataFrame()

for i in range(1, n_comp + 1):
    train_pca['pca_' + str(i)] = pca2_results_train[:, i - 1]

    train_ica['ica_' + str(i)] = ica2_results_train[:, i - 1]

    train_tsvd['tsvd_' + str(i)] = tsvd_results_train[:, i - 1]

    train_grp['grp_' + str(i)] = grp_results_train[:, i - 1]

    train_srp['srp_' + str(i)] = srp_results_train[:, i - 1]

    train_fag['fag_' + str(i)] = fag_results_train[:, i - 1]
    
dim_reds.append(('pca', train_pca))
dim_reds.append(('ica', train_ica))
dim_reds.append(('tsvd', train_tsvd))
dim_reds.append(('grp', train_grp))
dim_reds.append(('srp', train_srp))
dim_reds.append(('fag', train_fag))

In [20]:
# creating the combination from the '7' sets of reduced components
combs = [combinations(dim_reds, i+1) for i in range(0, len(dim_reds))]

dr_scores = list()
for c1 in combs:
    for c2 in c1:   
        train_, test_, id_ = list(), list(), list()
        for k in c2:
            train_.append(k[1])
            id_.append(k[0])
               
        train_x = train.reset_index(drop=True)
        train_.append(train_x)
            
        train_ = pd.concat(train_, axis=1)
        model = RandomForestRegressor()

        model.fit(train_, y_train)
        c_score = r2_score(y_train, model.predict(train_))
        
        dr_scores.append((','.join(id_), c_score))

        

NameError: name 'y_train' is not defined

In [19]:
rf = RandomForestRegressor()